In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [5]:
from datasets import load_dataset

arc_challenge = load_dataset("allenai/ai2_arc", "ARC-Challenge")
arc_easy = load_dataset("allenai/ai2_arc", "ARC-Easy")

README.md:   0%|          | 0.00/9.00k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/204k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/346k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/86.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

In [17]:
arc_challenge['validation'][298]

{'id': 'Mercury_7082145',
 'question': 'How should a line graph be used to display distance and time data for a moving object?',
 'choices': {'text': ['The y-axis should be labeled as time, which is the dependent variable.',
   'The y-axis should be labeled as distance, which is the independent variable.',
   'The x-axis should be labeled as distance, which is the dependent variable.',
   'The x-axis should be labeled as time, which is the independent variable.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D'}

In [10]:
import re
import ast

In [11]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList
from functools import reduce

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [13]:
import transformers
access_token = "YOUR_KEY"

In [14]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", token=access_token
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful and honest assistant. \
      Respond concisely and truthfully."},
    {"role": "user", "content": "Explain Einstein's relativity theory in three sentences."},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    num_return_sequences=2,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))
response1 = outputs[1][input_ids.shape[-1]:]
print(tokenizer.decode(response1, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Albert Einstein's theory of relativity is composed of two main components: special relativity and general relativity. Special relativity, introduced in 1905, states that the laws of physics are the same for all observers in uniform motion and that the speed of light is always constant, regardless of the observer's frame of reference. General relativity, introduced in 1915, expands on this idea by describing gravity as the curvature of spacetime caused by massive objects, which affects not only objects with mass but also the passage of time itself.
Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, introduced in 1905, posits that the laws of physics are the same for all observers in uniform motion relative to one another, and that the speed of light is always constant, regardless of the observer's frame of reference. General relativity, introduced in 1915, expands on this idea by describing gravity as th

In [18]:
def Answer_Agent_IO(question='', candidates='', num_seq=1):
    message = \
    f"""Given a question, give out the answer from the candidates, without any explanations. \
    Here is an example as guideline:
    "Question: What is the capital of France? \
    Candidates: A) London; B) Paris; C) Berlin; D) Madrid; \
    Answer: ###B###" \
    Make sure to include your answer in the format: ###your choice###. \
    Question: {question} \
    Candidates: {candidates} \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses

In [34]:
def Answer_Agent_CoT_0Shot(question='', candidates='', num_seq=1):
    message = \
    f"""Given a question, give out the answer from the candidates, without any explanations. \
    Make sure to include your answer in the format: ###your choice###. \
    Question: {question} \
    Candidates: {candidates} \
    Let's think step by step. \
    Answer: \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [24]:
ord('A')

65

In [33]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_IO(ori_question, choices_text, 1)[0]
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


In [35]:
all_answer_lm3_text_cot_0s = []
all_counts = -1

for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_CoT_0Shot(ori_question, choices_text, 1)[0]
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [36]:
import sys
import re
import string
from collections import Counter
import pickle

In [32]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [36]:
with open('answers_lm3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [48]:
with open('answers_lm3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [37]:
with open('answers_lm3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [40]:
with open('answers_lm3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [38]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answerKey'] for s in list(arc_challenge['validation'])[0:100]))

In [6]:
with open('answers_golden.txt', 'r') as file:
    golden = file.readlines()

In [7]:
with open('answers_lm3_io.txt', 'r') as file:
    output_io = file.readlines()

In [8]:
with open('answers_lm3_cot_0Shot.txt', 'r') as file:
    output_cot = file.readlines()

In [12]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)

    for (o, g) in zip(output, golden):
        o = o[0]
        g = g[0]
        if o == g:
            correct += 1
    
    return correct, correct/length


In [13]:
evaluation(output_io, golden)

(81, 0.81)

In [14]:
evaluation(output_cot, golden)

(84, 0.84)

In [ ]:
def Answer_Agent_debate(question='', prior=[], others='', num_seq=1):
    message = \
    f"""Given a question, and the reasoning processes from another agent, recheck your thoughts and answer. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Reasoning from another agent: {others} \
    Thought: \
    """
    
    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    for p in prior:
        messages.append({"role": "user", "content": p})
    messages.append({"role": "user", "content": message})
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses